In [7]:
import gymnasium as gym
import numpy as np

In [2]:
env = gym.make("CartPole-v1", render_mode = "human")

In [43]:
max_obs_values = env.observation_space.high
min_obs_values = env.observation_space.low
print(f"Actual observation max values: {max_obs_values}")
print(f"Actual observation min values: {min_obs_values}")
# This is kinda a lot, and we dont need infinity values, we can change/ignore them
# and we can change radians to degree too

# lets change them, not sure about this yet
# max_obs_values = (max_obs_values[0], 20, np.rad2deg(max_obs_values[2]), 20)
# min_obs_values = (min_obs_values[0], -20, np.rad2deg(min_obs_values[2]), -20)

# lets ignore them, simpler observations
# only position and angle is used
max_obs_values = np.array([max_obs_values[0], np.rad2deg(max_obs_values[2])])
min_obs_values = np.array([min_obs_values[0], np.rad2deg(min_obs_values[2])])
print(f"Used observation max values: {max_obs_values}")
print(f"Used observation min values: {min_obs_values}")

Actual observation max values: [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
Actual observation min values: [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
Used observation max values: [ 4.8 24. ]
Used observation min values: [ -4.8 -24. ]


In [59]:
DISCRETE_OBS_SPACE_SIZE = [20]* len(max_obs_values) # num_bins
DISCRETE_OBS_SPACE_SIZE = [20, 30]
discrete_obs_space_step_size = (max_obs_values - min_obs_values) / DISCRETE_OBS_SPACE_SIZE
discrete_obs_space_step_size

array([0.48000002, 2.4       ])

In [65]:
def discritizer(obs):
    obs = np.array([obs[0], obs[2]])
    discrete_obs = (obs - min_obs_values)/discrete_obs_space_step_size
    return discrete_obs
discritizer([-4.8, 2, 24, 6]) 
# min value corresponds to zero
# max value corresponds to discrete space size for that index

array([3.97364283e-07, 2.00000000e+01])

In [ ]:
DISCRETE_SIZE = ()

def discretizer(observation):
    return [0, 1]

In [ ]:
# num_episodes = 1
# for e in range(num_episodes):
#     observation, info = env.reset()
#     current_state = discretizer(*observation) 
#     done = False; score = 0
#     while not done:
#         # action = choose_action(current_state)
#         # if exploration_rate(e) > np.random.random():
#         action = env.action_space.sample() 

#         observation, reward, done, _, _ = env.step(action)
#         new_state = discretizer(*observation)

#         # update
#         # learnt_value = agent.new_Q_value(reward , new_state )
#         # old_value = agent.Q_table[current_state][action]
#         # agent.Q_table[current_state][action] = (1-agent.lr)*old_value + agent.lr*learnt_value


#         current_state = new_state
#         score += reward
#         env.render()

#         # print(current_state, observation)
#     print(f"Episode {e}, score {score}")

# env.close()


NameError: name 'discretizer' is not defined

: 